In [3]:
import layoutparser as lp
model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_R_50_FPN_3x/config')

In [14]:
import pymupdf
from PIL import Image

doc = pymupdf.open("../data/1706.03762.pdf")
page = doc[1].get_pixmap(dpi=300)
image = Image.frombytes("RGB", (page.width, page.height), page.samples)
print(image)



<PIL.Image.Image image mode=RGB size=2550x3300 at 0x1B92B3471D0>


In [ ]:
layout = model.detect(image)
lp.draw_box(image, layout)